# Import

In [ ]:
import pandas as pd
import numpy as np
import os
import re
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB


# data

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mksaad/arabic-sentiment-twitter-corpus")

print("Path to dataset files:", path)


Path to dataset files: /root/.cache/kagglehub/datasets/mksaad/arabic-sentiment-twitter-corpus/versions/2


In [ ]:
path = "/root/.cache/kagglehub/datasets/mksaad/arabic-sentiment-twitter-corpus/versions/2"
os.listdir(path)

['train_Arabic_tweets_negative_20190413.tsv',
 'train_Arabic_tweets_positive_20190413.tsv',
 'arabic_tweets',
 'test_Arabic_tweets_positive_20190413.tsv',
 'test_Arabic_tweets_negative_20190413.tsv']

In [ ]:
path = "/root/.cache/kagglehub/datasets/mksaad/arabic-sentiment-twitter-corpus/versions/2"

# Read all files and add sentiment labels
train_pos = pd.read_csv(f"{path}/train_Arabic_tweets_positive_20190413.tsv", sep="\t", header=None, names=["tweet"])
train_pos["sentiment"] = "positive"

train_neg = pd.read_csv(f"{path}/train_Arabic_tweets_negative_20190413.tsv", sep="\t", header=None, names=["tweet"])
train_neg["sentiment"] = "negative"

test_pos = pd.read_csv(f"{path}/test_Arabic_tweets_positive_20190413.tsv", sep="\t", header=None, names=["tweet"])
test_pos["sentiment"] = "positive"

test_neg = pd.read_csv(f"{path}/test_Arabic_tweets_negative_20190413.tsv", sep="\t", header=None, names=["tweet"])
test_neg["sentiment"] = "negative"

display(train_pos.head())
display(train_neg.head())

,tweet,sentiment
pos,نحن الذين يتحول كل ما نود أن نقوله إلى دعاء لل...,positive
pos,وفي النهاية لن يبقىٰ معك آحدإلا من رأىٰ الجمال...,positive
pos,من الخير نفسه 💛,positive
pos,#زلزل_الملعب_نصرنا_بيلعب كن عالي الهمه ولا ترض...,positive
pos,الشيء الوحيد الذي وصلوا فيه للعالمية هو : المس...,positive


,tweet,sentiment
neg,اعترف ان بتس كانو شوي شوي يجيبو راسي لكن اليوم...,negative
neg,توقعت اذا جات داريا بشوفهم كاملين بس لي للحين ...,negative
neg,#الاهلي_الهلال اكتب توقعك لنتيجة لقاء الهلال و...,negative
neg,نعمة المضادات الحيوية . تضع قطرة💧مضاد بنسلين ع...,negative
neg,الدودو جايه تكمل علي 💔,negative


merge-data


In [ ]:
df = pd.concat([train_pos, train_neg, test_pos, test_neg], ignore_index=True)
display(df.head())

,tweet,sentiment
0,نحن الذين يتحول كل ما نود أن نقوله إلى دعاء لل...,positive
1,وفي النهاية لن يبقىٰ معك آحدإلا من رأىٰ الجمال...,positive
2,من الخير نفسه 💛,positive
3,#زلزل_الملعب_نصرنا_بيلعب كن عالي الهمه ولا ترض...,positive
4,الشيء الوحيد الذي وصلوا فيه للعالمية هو : المس...,positive


# Data cleaning

In [ ]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

arabic_stopwords = set(stopwords.words("arabic"))

def clean_arabic_advanced(text):
    text = str(text)

    # 1) إزالة الروابط
    text = re.sub(r"http\S+|www.\S+", "", text)

    # 2) إزالة المنشن
    text = re.sub(r"@\w+", "", text)

    # 3) إزالة الهاشتاج بدون الكلمة
    text = re.sub(r"#", "", text)

    # 4) إزالة الأرقام
    text = re.sub(r"\d+", "", text)

    # 5) إزالة الرموز التعبيرية (Emoji)
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols
        "\U0001F680-\U0001F6FF"  # transport
        "\U0001F1E0-\U0001F1FF"  # flags
        "]+", flags=re.UNICODE)
    text = emoji_pattern.sub("", text)

    # 6) إزالة التشكيل (الفتحة ـ الكسرة ـ الضمة)
    tashkeel = re.compile("[\u064B-\u0652]")
    text = re.sub(tashkeel, "", text)

    # 7) إزالة التطويل (ـــــ)
    text = re.sub(r"ـ+", "", text)

    # 8) Normalization توحيد الحروف
    text = re.sub(r"[إأآا]", "ا", text)
    text = re.sub(r"ى", "ي", text)
    text = re.sub(r"ؤ", "و", text)
    text = re.sub(r"ئ", "ي", text)
    text = re.sub(r"ة", "ه", text)

    # 9) إزالة الحروف المتكررة (زي كداااااا -> كدا)
    text = re.sub(r'(.)\1{2,}', r'\1', text)

    # 10) حذف أي رمز غير عربي
    text = re.sub(r"[^؀-ۿ ]+", "", text)

    # 11) إزالة علامات الترقيم عربي + إنجليزي
    punct = r"[!\"#$%&'()*+,\-./:;<=>?@[\]^_`{|}~؟،]"
    text = re.sub(punct, " ", text)

    # 12) حذف المسافات الزائدة
    text = re.sub(r"\s+", " ", text).strip()

    # 13) إزالة الـ stopwords العربية
    words = text.split()
    words = [w for w in words if w not in arabic_stopwords]

    return " ".join(words)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


encoding


In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df["sentiment"] = le.fit_transform(df["sentiment"])

toknization

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

MAX_WORDS = 20000
MAX_LEN = 40

df["clean"] = df["tweet"].apply(clean_arabic_advanced)

tokenizer = Tokenizer(num_words=MAX_WORDS)
tokenizer.fit_on_texts(df["clean"])

X = tokenizer.texts_to_sequences(df["clean"])
X = pad_sequences(X, maxlen=MAX_LEN)

y = df["sentiment"]

# Spliting

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [ ]:
print(type(X_train))
print(X_train.shape)
print(X_train[0])


<class 'numpy.ndarray'>
(45436, 40)
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0 7663   36 1466 2792 1199 7664
 7665  360 7666  422   38  120   43 6773 1533  102 5010 8376]


# LSTM model


In [ ]:
MAX_WORDS = 30000
MAX_LEN = 60

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam

model = Sequential()

# Embedding Layer
model.add(Embedding(input_dim=MAX_WORDS, output_dim=200, input_length=MAX_LEN))

# Stacked BiLSTM Layers
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.4))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.4))

# Dense Layer
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.3))

# Output Layer
model.add(Dense(1, activation="sigmoid"))

# Compile Model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model.summary()





Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_3 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

train

In [ ]:
history = model.fit(
    X_train, y_train,
    batch_size=128,
    epochs=20,
    validation_split=0.2,
    callbacks=[es]
)


Epoch 1/20
284/284 ━━━━━━━━━━━━━━━━━━━━ 7s 24ms/step - accuracy: 0.8478 - loss: 0.3308 - val_accuracy: 0.7698 - val_loss: 0.4732
Epoch 2/20
284/284 ━━━━━━━━━━━━━━━━━━━━ 7s 24ms/step - accuracy: 0.8974 - loss: 0.2268 - val_accuracy: 0.7720 - val_loss: 0.5879
Epoch 3/20
284/284 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - accuracy: 0.9239 - loss: 0.1675 - val_accuracy: 0.7710 - val_loss: 0.8337
Epoch 4/20
284/284 ━━━━━━━━━━━━━━━━━━━━ 7s 24ms/step - accuracy: 0.9351 - loss: 0.1334 - val_accuracy: 0.7668 - val_loss: 0.8692


evaluation


In [ ]:
loss, acc = model.evaluate(X_test, y_test)
print("Test Accuracy:", acc)


355/355 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7699 - loss: 0.4555
Test Accuracy: 0.7752442955970764
